In [300]:
import pandas as pd
import numpy as np

In [301]:
data = pd.read_csv('./20181127_d4_ballotimage.txt', sep=" ", header=None)
print(data.head(15))

                                                0
0   000003600000918700000010020000507001000014200
1   000003600000918700000010020000507002000013800
2   000003600000918700000010020000507003000013900
3   000003600000918800000010020000507001000014200
4   000003600000918800000010020000507002000013700
5   000003600000918800000010020000507003000014000
6   000003600000918900000010020000507001000014100
7   000003600000918900000010020000507002000000001
8   000003600000918900000010020000507003000000001
9   000003600000919000000010020000507001000014200
10  000003600000919000000010020000507002000013700
11  000003600000919000000010020000507003000014000
12  000003600000919100000010020000507001000014200
13  000003600000919100000010020000507002000000001
14  000003600000919100000010020000507003000000001


In [302]:
ballot = data.iloc[:, 0].astype(str)
print(type(ballot.str[0:4]))


<class 'pandas.core.series.Series'>


In [303]:
ballot_data = pd.DataFrame({
    'district': ballot.str[0:7],
    'serial_number': ballot.str[7:16],
    'precint_number': ballot.str[16:23],
    'unsure_1': ballot.str[23: 26],
    'unsure_1': ballot.str[26: 33],
    'rcv': ballot.str[33: 36],
    'candidate_number': ballot.str[36: 43],
    'unsure_2': ballot.str[43:45]
})


In [304]:
pd.pivot_table(ballot_data, values='candidate_number', columns="rcv", index="serial_number", aggfunc="first")


rcv,001,002,003
serial_number,,,
000009187,0000142,0000138,0000139
000009188,0000142,0000137,0000140
000009189,0000141,0000000,0000000
000009190,0000142,0000137,0000140
000009191,0000142,0000000,0000000
000009192,0000139,0000000,0000000
000009193,0000142,0000139,0000144
000009194,0000144,0000143,0000137
000009195,0000144,0000138,0000142
